<a href="https://colab.research.google.com/github/vasiliyeskin/bentrevett-pytorch-seq2seq_ru/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Sequence to Sequence Learning with Neural Networks

В этой статье из серии мы построим модель машинного обучения для преобразования одной последовательности в другую, используя PyTorch и torchtext. Это будет продемонстрировано на примере переводов с немецкого на английский. Однако эта модель и все последующие могут быть применены к любой проблеме, которая предполагает преобразование одной последовательности в другую, например, для задачи обобщения, то есть преобразования длинной последовательности в более короткую на том же языке, или к задаче предсказания пространственной третичной структуры белков на основе последовательности нуклеотидной цепочки мРНК ([AlphaFold](https://deepmind.com/blog/article/alphafold-a-solution-to-a-50-year-old-grand-challenge-in-biology)).

В этой статье-скрипте для понимания общих концепций моделей seq2seq мы начнем с простого примера на основе модели из статьи [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215).

## Введение

Наиболее распространенными моделями sequence-to-sequence (seq2seq) являются модели *кодера-декодера*, которые обычно используют *рекуррентную нейронную сеть* (RNN) для *кодирования* исходного (входного) предложения в один вектор. Здесь и далее в серии мы будем называть этот единственный вектор *вектором контекста*. Мы можем думать о векторе контекста как об абстрактном представлении всего входного предложения. С точки зрения когнитивистов, этот вектор контекста — набор образов сущностей с образами взаимоотношений между ними. Затем данный вектор *декодируется* второй RNN, которая учится выводить целевое (выходное) предложение, генерируя его слово за словом, по одной ликсеме за раз.

![](https://github.com/vasiliyeskin/bentrevett-pytorch-seq2seq_ru/blob/master/assets/seq2seq1.png?raw=1)

На изображении выше показан пример перевода. Предложение ввода/источника, «guten morgen», проходит через слой эмбеддинга (желтый, служит для сопоставления элементов речи (слова, предложения,...) числовому вектору), а затем вводится в кодировщик (зеленый). Всегда добавляются токены *начало последовательности* (`<sos>` — *start of sequence*) и *конец последовательности* (`<eos>` — *end of sequence*) в начало и конец предложения соответственно. На каждом временном шаге на вход в RNN кодировщика подаётся эмбеддинг-версия текущего слова, $ e (x_t)$, которая порождена слоем эмбеддинга $ e $, так и скрытое состояние из предыдущего временного шага, $ h_{t-1} $. На выход RNN кодировщика подаёт новое скрытое состояние $ h_t $. Здесь мы можем думать о скрытом состоянии как о векторном представлении предложения. RNN кодировщика может быть представлена как функция (EncoderRNN) от переменных $ e (x_t) $ и $ h_ {t-1} $:

$$h_t = \text{EncoderRNN}(e(x_t), h_{t-1})$$

Здесь и далее используя аббревиатуру RNN, подразумеваем под ней сеть любой рекуррентной архитектуры: например, *LSTM* (Long Short-Term Memory) или *GRU* (Gated Recurrent Unit).

Здесь мы имеем $X = \{x_1, x_2, ..., x_T \}$, где $ x_1 = \text{<sos>}, x_{2} = \text{guten}$ и т.д. Начальное скрытое состояние, $h_0$, обычно либо инициализируется нулями, либо является параметром обучения.

После того как последнее слово, $ x_T $, было передано в RNN через слой эмбеддинга, мы используем конечное скрытое состояние, $ h_T $, как вектор контекста, то есть $ z = h_T $. Это векторное представление всего исходного предложения.

Теперь у нас есть вектор контекста $ z $, который мы можем начать декодировать, чтобы получить выходное/целевое предложение «доброе утро». Как и в предыдущем случае, мы добавляем токены начала и конца последовательности к целевому предложению. На каждом временном шаге входом в RNN декодера (синий) является эмбеддинг текущего слова, $ d (y_t) $, а также скрытое состояние из предыдущего временного шага, $ s_ { t-1} $, где начальное скрытое состояние декодера, $ s_0 $, является вектором контекста, $ s_0 = z $. Т.е. начальное скрытое состояние декодера является окончательным скрытым состоянием кодера. Таким образом, аналогично кодеру, мы можем представить декодер как функцию от переменных $ d (y_t) $ и s_ { t-1}:

$$s_t = \text{DecoderRNN}(d(y_t), s_{t-1})$$

Хотя слой эмбеддинга входа/источника $e$ и слой эмбеддинга выхода $d$ оба показаны жёлтым цветом на приведённом рисунке, они представляют собой два разных слоя эмбеддинга со своими собственными параметрами.

В декодере нам нужно перейти от скрытого состояния к фактическому слову, поэтому на каждом временном шаге мы используем $ s_t $ для распознавания (передавая эту величину через слой `Linear`, показанный фиолетовым цветом) того, что мы считаем следующим словом в последовательности $ \hat{y} _t $.

$$\hat{y}_t = f(s_t)$$

Слова в декодере всегда генерируются одно за другим, по одному за один временной шаг. Мы всегда используем `<sos>` в качестве первого слова $ y_1 $ для ввода в декодер, но для последующих слов для ввода, $ y_ {t> 1} $, мы иногда будем использовать фактическое, истинное/целевое следующее слово в последовательности $ y_t $, и иногда будем брать слово, предложенное нашим декодером, $ \hat{y} _ {t-1} $. Такой метод обучения называется *обучением с принуждением*, подробнее об этом [здесь](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/).

При обучении/тестировании нашей модели мы всегда знаем, сколько слов в нашем целевом предложении, поэтому мы перестаем генерировать слова, как только набираем это количество. Во время логического вывода обычно генерируют слова до тех пор, пока модель не выдаст токен окончания последовательности <eos> или после того, как будет сгенерировано определенное количество слов.

Когда у нас есть предсказанное предложение, $\hat{Y} = \{ \hat{y}_1, \hat{y}_2, ..., \hat{y}_T \}$, мы сравниваем его с нашим фактическим целевым предложение, $Y = \{ y_1, y_2, ..., y_T \}$, для расчета потерь/ошибок. Затем мы используем эти потери для обновления всех параметров обучаемой модели.

## Подготовка данных

Мы будем описывать модель в PyTorch и использовать torchtext для выполнения всей необходимой предварительной обработки. Кроме того, мы будем использовать spaCy для помощи в токенизации данных.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

Мы установим случайные начальные числа для получения детерминированных результатов.

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Далее мы создадим токенизаторы. Токенизатор используется для превращения строки, содержащей предложение, в список отдельных токенов, составляющих эту строку, например "доброе утро!" становится ["доброе", "утро", "!"]. С этого момента мы начнем говорить о предложениях, представляющих собой последовательность токенов, вместо того, чтобы говорить, что они являются последовательностью слов. Какая разница? Что ж, «добро» и «утро» - это и слова, и знаки, но «!» это знак, а не слово.

В spaCy есть модель для каждого языка (de_core_news_sm для немецкого и en_core_web_sm для английского), которую необходимо загрузить, чтобы мы могли получить доступ к токенизатору каждой модели.

**Примечание**: сначала необходимо загрузить модели на диск, используя следующую команду в командной строке:

```
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm
```


Для загрузки в Google Colab и для работы с версией Python 3.8 используем следующие команды (После загрузки обязательно перезапустите colab runtime! Наибыстрейший способ через короткую комаду： **Ctrl + M + .**):

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Мы загружаем модели в память следующим образом:

In [5]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

Далее мы создаем токенизирующие функции. Они могут быть переданы в torchtext и будут принимать предложение в виде строки, а возвращать предложение в виде списка токенов.

В статье, на основе которой мы реализуем данную модель, авторы считают полезным изменить порядок ввода токенов, который, по их мнению, «вводит множество краткосрочных зависимостей в данные, значительно упрощая задачу оптимизации». Мы копируем результат, переворачивая немецкое предложение после того, как оно было преобразовано в список токенов.

In [6]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Класс `Field` в torchtext указывает как данные должны быть обработаны. Все возможные аргументы детально описаны [здесь](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L61).

Мы устанавливаем правильную функцию токенизации в качестве аргумента `tokenize` для каждого языка, причем немецкий является полем` SRC` (источник), а английский — полем `TRG` (цель). Экземпляры класса `Field` содержат токены «начало последовательности» и «конец последовательности» как аргументы «init_token» и «eos_token». Все слова преобразуются к нижнему регистру.

In [7]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

Далее скачиваем и загружаем данные для обучения, валидации во время обучения и тестирования.

Набор данных, который мы будем использовать, - это [набор данных Multi30k](https://github.com/multi30k/dataset). Это набор данных из ~ 30 000 параллельных предложений на английском, немецком и французском языках, каждое из которых содержит порядка 12-ти слов в предложении.

`exts` указывает, какие языки использовать в качестве источника и цели (источник идет первым), а `fields` указывает, какое поле использовать для источника и цели.

In [9]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

Мы можем проверить, что мы загрузили нужное количество примеров:

In [10]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


Мы также можем распечатать пример, убедившись, что исходное предложение перевернуто:

In [11]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


Точка стоит в начале предложения на немецком языке (src), поэтому похоже, что предложение было правильно перевернуто.

Затем мы создадим *словарь* для исходного и целевого языков. Словарь используется для связывания каждого уникального токена с индексом (целым числом). Словари исходного и целевого языков различаются.

Используя аргумент `min_freq`, мы разрешаем только токенам, которые появляются как минимум 2 раза, появляться в нашем словаре. Токены, которые появляются только один раз, конвертируются в токен `<unk>` (неизвестно).

Важно отметить, что наш словарь должен строиться только на основе обучающего набора, а не набора для проверки/тестирования. Это предотвращает «утечку информации» в нашу модель, давая нам искусственно завышенные оценки валидации/тестирования.

In [12]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [13]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


Последний шаг подготовки данных — создание итераторов. С их помощью можно итерационно возвращать пакеты данных, которые будут иметь атрибут `src` (тензоры PyTorch, содержащие набор оцифрованных исходных предложений) и атрибут `trg` (тензоры PyTorch, содержащие набор оцифрованных целевых предложений). Оцифрованные предложения — это просто причудливый способ сказать, что они были преобразованы из последовательности читаемых токенов в последовательность соответствующих индексов с использованием словаря.

Нам также нужно определить `torch.device`. Эта величина используется, чтобы указать torchText, куда помещать тензоры для вычислений: на GPU или нет? Мы используем функцию `torch.cuda.is_available()`, которая вернет `True`, если на нашем компьютере обнаружен графический процессор. Мы передаем это поле итератору.

Когда мы получаем батч примеров с использованием итератора, нам нужно убедиться, что все исходные предложения дополнены до одинаковой длины, как и целевые предложения. К счастью, итераторы torchText справятся с этим за нас!

Мы используем `BucketIterator` вместо стандартного `Iterator`, поскольку он создает пакеты таким образом, чтобы минимизировать количество отступов как в исходном, так и в целевом предложении.

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Построение модели Seq2Seq

Мы будем строить нашу модель из трех частей. Кодер, декодер и модель seq2seq, которая инкапсулирует кодер и декодер, и обеспечивает способ взаимодействия с каждым из них.

### Кодер

Во-первых, кодер — это двухслойная LSTM сеть. В статье, на которую мы опираемся при реализации, используется 4-слойная LSTM сеть, но в целях экономии времени мы сократили количество слоёв до 2-х. Многослойную RNN легко расширить с 2 до 4 уровней.

Во многослойной RNN входное предложение $X$ после эмбеддинга передаётся в первый (нижний) слой RNN и порождает скрытые состояния $H=\{h_1, h_2, ..., h_T\}$. Выходные данные этого слоя используются в качестве входных данных для RNN на уровне выше. Таким образом, представляя каждый слой надстрочным индексом, скрытые состояния в первом слое задаются следующим образом:

$$h_t^1 = \text{EncoderRNN}^1(e(x_t), h_{t-1}^1)$$

Скрытые состояния во втором слое задаются следующим образом:

$$h_t^2 = \text{EncoderRNN}^2(h_t^1, h_{t-1}^2)$$

Использование многослойной RNN предполагает, что нам понадобится исходное скрытое состояние в качестве входных данных для каждого слоя $h_0^l$, и сеть будет предоставлять вектор контекста для каждого слоя $z^l$.

Не вдаваясь в подробности о LSTM сети (см. [здесь](https://colah.github.io/posts/2015-08-Understanding-LSTMs/), чтобы узнать о них больше), все, что нам нужно знать, это то, что эти сети являются RNN, которые вместо того, чтобы просто принимать скрытое состояние и возвращать новое скрытое состояние для каждого временного шага, принимают и возвращают *состояние ячейки* $ c_t $, для каждого временного шага.

$$\begin{align*}
h_t &= \text{RNN}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})
\end{align*}$$

Мы можем принять $c_t$ как другой тип скрытого состояния. Подобно $h_0^l$, величина $c_0^l$ будет инициализирована тензором, заполненным нулями. Кроме того, наш вектор контекста теперь будет образован конечным скрытым состоянием и конечным состоянием ячейки, то есть $z^l = (h_T^l, c_T^l)$.

Распространяя наши многослойные уравнения на LSTM, мы получаем:

$$\begin{align*}
(h_t^1, c_t^1) &= \text{EncoderLSTM}^1(e(x_t), (h_{t-1}^1, c_{t-1}^1))\\
(h_t^2, c_t^2) &= \text{EncoderLSTM}^2(h_t^1, (h_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Обратите внимание на то, что наше скрытое состояние из первого слоя передается в качестве входных данных во второй уровень, а не состояние ячейки.

Итак, наш кодировщик выглядит примерно так:

![](https://github.com/vasiliyeskin/bentrevett-pytorch-seq2seq_ru/blob/master/assets/seq2seq2.png?raw=1)

Мы реализуем это в коде, создав модуль `Encoder`, который требует, чтобы мы унаследовали это класс от` torch.nn.Module` и использовали `super () .__ init __ ()` как некоторый шаблонный код. Кодировщик принимает следующие аргументы:
- `input_dim` - размер / размерность one-hot векторов, которые будут вводиться в кодировщик. Они равны размеру входного (исходного) размера словаря.
- `emb_dim` - размерность слоя эмбеддинга. Этот слой преобразует one-hot векторы в плотные векторы с размерами `emb_dim`.
- `hid_dim` - размерность скрытого состояния и состояния ячейки.
- `n_layers` - количество слоев в RNN.
- `dropout` - это количество использованных отсевов. Это параметр регуляризации для предотвращения переобучения. Ознакомьтесь с [этим здесь](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR) для получения дополнительных сведений о прекращении обучения.

Мы не собираемся подробно обсуждать слой эмбеддинга в этих заметках. Все, что нам нужно знать, это то, что есть шаг до того, как слова - технически индексы слов - передаются в RNN, где слова преобразуются в векторы. Чтобы узнать больше о встраивании слов, посмотрите эти статьи: [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/).

Слой эмбеддинга создаётся с помощью `nn.Embedding`, LSTM с `nn.LSTM` и слой дропаута с `nn.Dropout`. Дополнительные сведения об этом см. [документации](https://pytorch.org/docs/stable/nn.html).

Следует отметить, что аргумент `dropout` для LSTM заключается в том, сколько отсева необходимо применять между уровнями многослойной RNN, то есть между скрытыми состояниями, выводимыми из уровня $ l $, и теми же скрытыми состояниями, используемыми для ввод слоя $ l + 1 $.

В методе `forward` мы передаем исходное предложение $ X $, которое преобразуется в плотные векторы с помощью слоя `embedding`, а затем применяется дропаут. Эти величины через эмбеддинг затем передаются в RNN. Когда мы передаем всю последовательность в RNN, она автоматически выполняет для нас повторяющийся расчет скрытых состояний по всей последовательности! Обратите внимание, что мы не передаем в RNN начальное скрытое состояние или состояние ячейки. Это связано с тем, что, как указано в [документации] (https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM), если в RNN не передается скрытое состояние / состояние ячейки, оно автоматически создает начальное скрытое состояние / состояние ячейки как тензор всех нулей.

RNN возвращает: `выходные` (скрытые состояния верхнего уровня для каждого временного шага), `скрытые` (окончательное скрытое состояние для каждого слоя, $h_T$, наложенное друг на друга) и `ячейка`»` (конечное состояние ячейки для каждого слоя, $c_T$, наложенных друг на друга).

Поскольку нам нужны только окончательные скрытой состояния и состояние ячейки (для создания нашего вектора контекста), `forward` только возвращает `hidden` и `cell`.

Размеры каждого из тензоров оставлены в виде комментариев в коде. В этой реализации `n_directions` всегда будет 1, однако обратите внимание, что двунаправленные RNN (описанные в учебнике 3) будут иметь` n_directions` как 2.

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

### Декодер

Далее мы построим наш декодер, который как и кодер будет двухслойной LSTM сетью (4-х в статье).

![](https://github.com/vasiliyeskin/bentrevett-pytorch-seq2seq_ru/blob/master/assets/seq2seq3.png?raw=1)

Объект класса `Decoder` выполняет один шаг декодирования, то есть выводит один токен за каждую временную итерацию. Первый слой получает скрытое состояние и состояние ячейки из предыдущего временного шага $(s_{t-1}^1, c_{t-1}^1)$ и передаёт его через LSTM вместе с текущим встроенным токеном $y_t$, для создания нового скрытого состояния и состояния ячейки $(s_t^1, c_t^1)$. Последующие слои будут использовать скрытое состояние из слоя ниже, $s_t^{l-1}$, и предыдущие скрытые состояния и состояния ячеек из своего слоя, $(s_{t-1}^l, c_{t-1}^l)$. Это приводит к выражениям, очень похожими на те, что используются для кодировщика.

$$\begin{align*}
(s_t^1, c_t^1) = \text{DecoderLSTM}^1(d(y_t), (s_{t-1}^1, c_{t-1}^1))\\
(s_t^2, c_t^2) = \text{DecoderLSTM}^2(s_t^1, (s_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Помните, что начальные скрытые состояния и состояния ячеек для нашего декодера — это наши векторы контекста, которые являются конечными скрытыми состояниями и состояниями ячеек нашего кодировщика с того же уровня, т.е. $(s_0^l,c_0^l)=z^l=(h_T^l,c_T^l)$.

Затем мы передаём скрытое состояние из верхнего уровня RNN $s_t^L$, через линейный слой $f$, для предложения следующего токена в целевой (выходной) последовательности $\hat{y}_{t+1}$.

$$\hat{y}_{t+1} = f(s_t^L)$$

Аргументы и инициализация аналогичны классу `Encoder`, за исключением того, что теперь у нас есть `output_dim`, который является размером словаря для выходной/целевой последовательности. Кроме того, добавлен слой `Linear`, используемый для прогнозирования токена на основе скрытого состояния верхнего уровня.

Метод `forward` принимает пакет входных токенов, предыдущие скрытые состояния и предыдущие состояния ячеек. Поскольку мы декодируем только один токен за раз, входные токены всегда будут иметь единичную длину последовательности. Мы «разжимаем» входные токены для получения предложения длины 1. Затем, как и в кодировщике, мы проходим через слой эмбеддинга и применяем дропаут. Этот пакет встроенных токенов затем передается в RNN с предыдущими скрытыми состояниями и состояниями ячеек. Таким образом создаётся «вывод» (скрытое состояние из верхнего уровня RNN), новое «скрытое» состояние (по одному для каждого слоя, накладываемое друг на друга) и новое состояние «ячейки» (также по одному на слой, уложеные друг на друга). Затем мы передаем «вывод» (после того, как избавляемся от измерения длины предложения) через линейный слой, чтобы получить «предложенное» слово. Затем мы возвращаем  «предложенное» слово, новое «скрытое» состояние и новое состояние «ячейки».

**Примечание**: т.к. у нас всегда длина последовательности равна 1, мы могли бы использовать `nn.LSTMCell` вместо `nn.LSTM`, поскольку он предназначен для обработки пакета входных данных, которые необязательно входят в последовательность. nn.LSTMCell - это просто одна ячейка, а nn.LSTM - потенциально оболочка вокруг нескольких ячеек. Использование `nn.LSTMCell` в этом случае означало бы, что нам не нужно «разжимаеть» предложение, чтобы добавить измерение длины промежуточной последовательности, но нам понадобится один `nn.LSTMCell` на каждый слой в декодере и, чтобы гарантировать, что каждый` nn .LSTMCell` получает правильное начальное скрытое состояние от кодировщика. Все это делает код менее лаконичным — отсюда и решение придерживаться обычного `nn.LSTM`.

In [17]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

### Seq2Seq

В заключительной части мы реализуем модель seq2seq. Она будет:
- получать входные предложения (предложения из источника)
- использовать кодировщик для создания векторов контекста
- использовать декодер для создания выходного/целевого предложения

Наша полная модель будет выглядеть следующим образом:

![](https://github.com/vasiliyeskin/bentrevett-pytorch-seq2seq_ru/blob/master/assets/seq2seq4.png?raw=1)

Модель `Seq2Seq` принимает кодировщик, декодер и ссылку на девайс (используется для размещения тензоров на GPU, если он существует).

Для этой реализации мы должны убедиться, что количество слоев и скрытые (и ячейки) размеры равны в `Encoder` и `Decoder`. Это не всегда так, и нам необязательно нужно соблюдать равенство количества слоев или равенство размера скрытых измерений в модели sequence-to-sequence. Однако, если бы мы сделали количество слоёв отличными, нам нужно было бы принять решение о том, как обрабатывать результаты. Например, если у нашего кодировщика 2 слоя, а у нашего декодера только 1, как это обрабатывать? Усреднять ли мы два вектора контекста, выводимых декодером? Пропускать ли оба через линейный слой? Использовать ли только вектор контекста с самого верхнего уровня? И т.п.

Наш метод `forward` берет исходное предложение, целевое предложение и коэффициент, показывающий вероятность включения метода `обучения с принуждением`. При декодировании на каждом временном шаге мы будем предсказывать, какой следующий токен в целевой последовательности будет взят из предыдущих декодированных токенов, $\hat{y}_{t+1}=f(s_t^L)$. С вероятностью, равной указанноуму выше коэффициенту (`teacher_forcing_ratio`), мы будем использовать фактический следующий токен истинной/целеовй последовательности в качестве входных данных для декодера в течение следующего временного шага. Однако с вероятностью `1 - teacher_forcing_ratio` мы будем использовать токен, предсказанный моделью, в качестве следующего входа в модель, даже если он не соответствует фактическому следующему токену в истинной последовательности.

Первое, что мы делаем в методе `forward`, - это создаём тензор` output`, который будет хранить все предложения модели прогнозы $\hat{Y}$.

Затем мы загружаем входное предложение, `src`, в кодировщик и получаем окончательные скрытые состояния и состояния ячеек.

Первым входом в декодер является токен начала последовательности (`<sos>`). Поскольку к нашему тензору `trg` уже добавлен токен `<sos>` (определяли `init_token` в нашем поле `TRG`), мы получаем $y_1$, разрезая его. Мы знаем, какой длины должны быть наши целевые предложения (`max_len`), поэтому мы повторяем этот цикл необходимое количество раз. Последний токен, введенный в декодер, - это токен перед токеном `<eos>` - токен `<eos>` никогда не вводится в декодер.

Во время каждой итерации цикла мы:
- передаём входное предложение, предыдущее скрытое и предыдущее состояния ячейки ($y_t, s_{t-1}, c_{t-1}$) в декодер
- получаем предсказание, следующее скрытое состояние и следующее состояние ячейки ($\hat{y}_{t+1}, s_{t}, c_{t}$) из декодера
- размещаем наше предсказание, $\hat{y}_{t+1}$/`output` в нашем тензоре предсказаний, $\hat{Y}$/`outputs`
- решаем использовать ли обучение с принуждением
    - если мы решаем использовать, то следующий `ввод` - это следующий токен в последовательности, $y_{t+1}$/`trg[t]`
    - если мы решаем не использовать, то `ввод` - это следующий токен в последовательности, $\hat{y}_{t+1}$/`top1`, который мы получаем, пропуская выходной тензор через `argmax`

После того как мы сделали все действия, мы возвращаем наш тензор, полный предсказаний $\hat{Y}$/`outputs`.

**Примечание**: наш цикл декодера начинается с 1, не с 0. Это означает, что в нулевом элементе нашего тензора "выходов" остаются все нули. Так что наши `trg` и` output` выглядят следующим образом:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Позже, когда мы вычисляем потери, мы отсекаем первый элемент каждого тензора, чтобы получить:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

# Обучение модели Seq2Seq

Теперь, когда наша модель готова, мы можем приступить к ее обучению.

Сначала мы инициализируем нашу модель. Как упоминалось ранее, размеры ввода и вывода определяются размером словаря. Размеры эмбеддинга и дропаута для кодера и декодера могут быть разными, но количество слоёв и размерность скрытых состояний должны быть одинаковыми.

Затем определяем кодер, декодер и нашу модель Seq2Seq, которую мы загружаем в `device`.


In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

Затем инициализируем веса нашей модели. В статье утверждается, что инициализируют все веса из равномерного распределения между -0.08 и +0.08, т.е. $\mathcal{U}(-0.08, 0.08)$.

Мы инициализируем веса в PyTorch, создавая функцию `apply`, которую мы применяем к нашей модели. При использовании функции `apply` функция весов `init_weights` будет вызываться для каждого модуля и подмодуля в нашей модели. Для каждого модуля мы перебираем все параметры и выбираем их из равномерного распределения с помощью `nn.init.uniform_`.

In [20]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Мы определяем функцию, которая будет вычислять количество обучаемых параметров в модели.

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,898,501 trainable parameters


Мы определяем наш оптимизатор, который используем для обновления наших параметров в цикле обучения. Посмотрите [здесь](http://ruder.io/optimizing-gradient-descent/) информацию о различных оптимизаторах. Здесь мы будем использовать оптимизатор `Adam`.

In [23]:
optimizer = optim.Adam(model.parameters())

Затем мы определяем нашу функцию потерь. Функция `CrossEntropyLoss` вычисляет как log softmax, так и отрицательную логарифмическую вероятность наших прогнозов.

Наша функция потерь вычисляет средние потери на токен, однако, передав индекс токена `<pad>` в качестве аргумента `ignore_index`, мы игнорируем потерю всякий раз, когда целевой токен является маркером заполнения `<pad>`.

In [24]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Далее мы определим наш цикл обучения.

Сначала мы установим модель в «режим обучения» с помощью `model.train ()`. Это включит дропаут (и пакетную нормализацию, которую мы не используем), а затем итерацию через наш итератор данных.

Как указывалось ранее, наш цикл декодера начинается с 1, а не с 0. Это означает, что в нулевом элементе нашего тензора `output` остаются все нули. Итак, наши `trg` и `output` выглядят примерно так:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Здесь, когда мы вычисляем потери, мы отсекаем первый элемент каждого тензора, чтобы получить:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

На каждой итерации:
- получаем исходное и целевое предложения из пакета, $X$ и $Y$
- обнуляем градиенты, вычисленные из последнего пакета
- передаём исходное и целевое предложения в модель, чтобы получить результат, $\hat{Y}$
- так как функция потерь работает только с двумерынм входом с одномерным целевым выходом, нам нужно `спрямить` каждую из них с помощью `.view`
    - мы отрезаем первый столбец выходных и целевых тензоров, как упомянуто выше
 - вычисляем градиенты с помощью` loss.backward () `
 - обрезаем градиенты, чтобы предотвратить их взрыв (обычная проблема в RNN)
 - обновляем параметры нашей модели, выполнив шаг оптимизатора
 - суммируем значения потерь в промежуточную сумму

Наконец, мы возвращаем потери, усредненные по всем батчам.

In [25]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Наш цикл оценки аналогичен нашему циклу обучения, однако, поскольку мы не обновляем какие-либо параметры, нам не нужно передавать оптимизатор или значение отсечки.

Мы должны не забыть установить модель в режим оценки с помощью `model.eval()`. Это отключит дропаути (и нормализацию партии, если она используется).

Мы используем блок `with torch.no_grad()`, чтобы гарантировать, что градиенты не вычисляются внутри блока. Это снижает потребление памяти и ускоряет работу.

Цикл итерации аналогичен (без обновлений параметров), однако мы должны убедиться, что мы отключили обучение с принуждением. Это приведет к тому, что модель будет использовать только свои собственные предположения, чтобы делать дальнейшие предположения для предложения, что отражает то, как она будет использоваться при развертывании.

In [26]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Затем мы создадим функцию, которую мы будем использовать, чтобы сообщить нам, сколько времени занимает эпоха.

In [27]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Наконец-то мы можем приступить к обучению нашей модели!

В каждую эпоху мы будем проверять, достигла ли наша модель лучших потерь на валидации на данный момент. Если это так, мы обновим величину, отражающую лучшие потери при валидации, и сохраним параметры нашей модели (называемой `state_dict` в PyTorch). Затем, когда мы приступим к тестированию нашей модели, мы будем использовать сохраненные параметры, используемые для достижения наилучших потерь при проверке.

Мы будем печатать и потери, и `сложность` (perplexity) на каждой эпохе. Легче увидеть изменение в `сложность`, чем изменение потерь, поскольку цифры намного больше.

In [28]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 36s
	Train Loss: 5.046 | Train PPL: 155.365
	 Val. Loss: 4.919 |  Val. PPL: 136.915
Epoch: 02 | Time: 0m 35s
	Train Loss: 4.461 | Train PPL:  86.555
	 Val. Loss: 4.751 |  Val. PPL: 115.679
Epoch: 03 | Time: 0m 33s
	Train Loss: 4.173 | Train PPL:  64.879
	 Val. Loss: 4.545 |  Val. PPL:  94.161
Epoch: 04 | Time: 0m 33s
	Train Loss: 3.974 | Train PPL:  53.209
	 Val. Loss: 4.478 |  Val. PPL:  88.059
Epoch: 05 | Time: 0m 33s
	Train Loss: 3.827 | Train PPL:  45.909
	 Val. Loss: 4.407 |  Val. PPL:  82.027
Epoch: 06 | Time: 0m 33s
	Train Loss: 3.697 | Train PPL:  40.319
	 Val. Loss: 4.274 |  Val. PPL:  71.821
Epoch: 07 | Time: 0m 33s
	Train Loss: 3.576 | Train PPL:  35.715
	 Val. Loss: 4.164 |  Val. PPL:  64.341
Epoch: 08 | Time: 0m 33s
	Train Loss: 3.445 | Train PPL:  31.353
	 Val. Loss: 4.102 |  Val. PPL:  60.442
Epoch: 09 | Time: 0m 33s
	Train Loss: 3.325 | Train PPL:  27.806
	 Val. Loss: 4.019 |  Val. PPL:  55.638
Epoch: 10 | Time: 0m 33s
	Train Loss: 3.252 | Train PPL

Мы загрузим параметры (`state_dict`), которые были сохранены для лучшей модели при проверке, и запустим их на тестовом наборе.

In [29]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.047 | Test PPL:  57.198 |


В следующей статье мы реализуем модель, которая позволяет повысить сложность тестирования, но использует только один уровень в кодировщике и декодере.

Вывод предоставленного оборудования и библиотек для Google Colab

In [ ]:
!nvidia-smi
!pip list


Fri Jun  4 20:46:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------